# Redes Neuronales (solo con numpy)

In [1]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['figure.dpi']=300

from mpl_toolkits.axes_grid1.inset_locator import inset_axes

## Funciones

In [2]:
def apply_layer(y_in,w,b,activation):
    """
    Calcula el paso de una capa a la siguiente, dados
    una matriz de pesos w (tamano [n_neurons_in,n_neurons_out])
    un vector bias b (longitud n_neurons_out)
    y los valores de la neurona de entrada y_in 
    (tamano [batchsize,n_neurons_in])
    
    retorna los valores de las neuronas de salida en la siguiente capa
    (tamano [batchsize, n_neurons_out])
    """
    z=np.dot(y_in, w) + b # proceso en lote: y_in es de tamano [batchsize,num_neurons_in]

    if activation=='sigmoid':
        return(1/(1 + np.exp(-z)))
    elif activation=='jump':
        return(np.array(z>0, dtype='float'))
    elif activation=='linear':
        return(z)
    elif activation=='reLU':
        return((z>0)*z)

In [3]:
def apply_net(y_in,weights,biases,activations):
    """
    Aplica toda una red neuronal de multiples capas
    """
    y = y_in
    for j in range(len(biases)):
        y = apply_layer(y, weights[j], biases[j], activations[j])
    return(y)

In [4]:
# algunas rutinas internas para hacer el plot de la red
def plot_connection_line(ax, X, Y, W, vmax = 1.0, linewidth = 3):
    t = np.linspace(0, 1, 20)
    if W>0:
        col=[0, 0.4, 0.8]
    else:
        col=[1, 0.3, 0]
    ax.plot(X[0] + (3*t**2 - 2*t**3)*(X[1] - X[0]), Y[0] + t*(Y[1] - Y[0]),
           alpha = abs(W) / vmax, color = col,
           linewidth = linewidth)

In [5]:
def plot_neuron_alpha(ax, X, Y, B, size = 100.0, vmax = 1.0):
    if B>0:
        col=[0, 0.4, 0.8]
    else:
        col=[1, 0.3, 0]
    ax.scatter([X], [Y], marker = 'o', c = col, alpha = abs(B) / vmax, s = size, zorder = 10)

In [6]:
def plot_neuron(ax, X, Y, B, size = 100.0, vmax = 1.0):
    if B>0:
        col=[0, 0.4, 0.8]
    else:
        col=[1, 0.3, 0]
    ax.scatter([X], [Y], marker = 'o', c = col, s = size, zorder = 10)

In [ ]:
def visualize_network(weights, biases, activations,
                      M = 100, y0range = [-1, 1], y1range = [-1, 1],
                     size = 400.0, linewidth = 5.0):
    """
    Visualize a neural network with 2 input 
    neurons and 1 output neuron (plot output vs input in a 2D plot)
    
    weights is a list of the weight matrices for the
    layers, where weights[j] is the matrix for the connections
    from layer j to layer j+1 (where j==0 is the input)
    
    weights[j][m,k] is the weight for input neuron k going to output neuron m
    (note: internally, m and k are swapped, see the explanation of
    batch processing in lecture 2)
    
    biases[j] is the vector of bias values for obtaining the neurons in layer j+1
    biases[j][k] is the bias for neuron k in layer j+1
    
    activations is a list of the activation functions for
    the different layers: choose 'linear','sigmoid',
    'jump' (i.e. step-function), and 'reLU'
    
    M is the resolution (MxM grid)
    
    y0range is the range of y0 neuron values (horizontal axis)
    y1range is the range of y1 neuron values (vertical axis)
    """
    swapped_weights=[]
    for j in range(len(weights)):
        swapped_weights.append(np.transpose(weights[j]))
        
    y0,y1=np.meshgrid(np.linspace(y0range[0],y0range[1],M),np.linspace(y1range[0],y1range[1],M))
    y_in=np.zeros([M*M,2])
    y_in[:,0]=y0.flatten()
    y_in[:,1]=y1.flatten()
    y_out=apply_net(y_in,swapped_weights,biases,activations)

    fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(8,4))
    
    # plot the network itself:
    
    # positions of neurons on plot:
    posX=[[-0.5,+0.5]]; posY=[[0,0]]
    vmax=0.0 # for finding the maximum weight
    vmaxB=0.0 # for maximum bias
    for j in range(len(biases)):
        n_neurons=len(biases[j])
        posX.append(np.array(range(n_neurons))-0.5*(n_neurons-1))
        posY.append(np.full(n_neurons,j+1))
        vmax=np.maximum(vmax,np.max(np.abs(weights[j])))
        vmaxB=np.maximum(vmaxB,np.max(np.abs(biases[j])))

    # plot connections
    for j in range(len(biases)):
        for k in range(len(posX[j])):
            for m in range(len(posX[j+1])):
                plot_connection_line(ax[0],[posX[j][k],posX[j+1][m]],
                                     [posY[j][k],posY[j+1][m]],
                                     swapped_weights[j][k,m],vmax=vmax,
                                    linewidth=linewidth)
    
    # plot neurons
    for k in range(len(posX[0])): # input neurons (have no bias!)
        plot_neuron(ax[0],posX[0][k],posY[0][k],
                   vmaxB,vmax=vmaxB,size=size)
    for j in range(len(biases)): # all other neurons
        for k in range(len(posX[j+1])):
            plot_neuron(ax[0],posX[j+1][k],posY[j+1][k],
                       biases[j][k],vmax=vmaxB,size=size)
    
    ax[0].axis('off')
    
    # now: the output of the network
    img=ax[1].imshow(np.reshape(y_out,[M,M]),origin='lower',
                    extent=[y0range[0],y0range[1],y1range[0],y1range[1]])
    ax[1].set_xlabel(r'$y_0$')
    ax[1].set_ylabel(r'$y_1$')
    
    axins1 = inset_axes(ax[1],
                    width="40%",  # width = 50% of parent_bbox width
                    height="5%",  # height : 5%
                    loc='upper right')

    imgmin=np.min(y_out)
    imgmax=np.max(y_out)
    color_bar=fig.colorbar(img, cax=axins1, orientation="horizontal",ticks=np.linspace(imgmin,imgmax,3))
    cbxtick_obj = plt.getp(color_bar.ax.axes, 'xticklabels')
    plt.setp(cbxtick_obj, color="white")
    axins1.xaxis.set_ticks_position("bottom")

    plt.show()